<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/9_Finans_i%C3%A7in_Python_Oto_Fibonacci.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Kütüphanelerin İndirilmesi
!pip install git+https://github.com/rongardF/tvdatafeed

#Kütüphanelerin Yüklenmesi
import numpy as np
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from scipy.signal import argrelextrema

# TvDatafeed in çağırılması
tv = TvDatafeed()

#Hisse İsimlerinin Tanımlanması
bist100 = [
        "AEFES","AGHOL","AGROT","AKBNK","AKFGY","AKFYE","AKSA","AKSEN","ALARK","ALFAS",
        "ARCLK","ARDYZ","ASELS","ASTOR","BERA","BFREN","BIMAS","BINHO","BRSAN","BRYAT",
        "BTCIM","CANTE","CCOLA","CIMSA","CWENE","DOAS","DOHOL","ECILC","ECZYT","EGEEN",
        "EKGYO","ENERY","ENJSA","ENKAI","EREGL","EUPWR","EUREN","FROTO","GARAN","GESAN",
        "GOLTS","GUBRF","HALKB","HEKTS","ISCTR","ISGYO","ISMEN","IZENR","KAYSE","KCAER",
        "KCHOL","KLSER","KONTR","KONYA","KOZAA","KOZAL","KRDMD","KTLEV","LMKDC","MAVI",
        "MGROS","MIATK","OBAMS","ODAS","OTKAR","OYAKC","PEKGY","PETKM","PGSUS","QUAGR",
        "REEDR","SAHOL","SASA","SDTTR","SISE","SKBNK","SMRTG","SOKM","TABGD","TAVHL",
        "TCELL","THYAO","TKFEN","TKNSA","TMSN","TOASO","TSKB","TTKOM","TTRAK","TUKAS",
        "TUPRS","TURSG","ULKER","VAKBN","VESBE","VESTL","YEOTK","YKBNK","YYLGD","ZOREN"
    ]

#Bist100 Veri Kaydı için Dataframe Oluşturulması
df_bist=pd.DataFrame(columns=["datetime","Hisse","val","type","Fibo0","Fibo236","Fibo382","Fibo618","Fibo100",
                              "Range","Pct_Fibo0","Pct_Fibo236","Pct_Fibo382","Pct_Fibo618","Pct_Fibo100"])

#Fibonachi Seviyelerinin hesaplanması için kullanılacak fonksiyon
def calc_fibo(X, Y):
    A = min(X, Y)
    B = max(X, Y)
    Fibo3 = B - (B - A) * 0.382
    Fibo2 = B - (B - A) * 0.618
    Fibo1 = B - (B - A) * 0.764
    Fibo_List = [A, Fibo1, Fibo2, Fibo3, B]
    return Fibo_List

#Uzaklığın hesaplanması için kullanılacak fonksiyon
def calculate_percentage_distance(current, levels):
    percentages = [(current - level) / current * 100 for level in levels]
    percentages = [round(abs(percentage), 2) for percentage in percentages]
    return percentages

# Verilerin İşlenmesi
for Hisse in bist100:
    try:
        #Datanın çekilmesi
        data = tv.get_hist(symbol=Hisse, exchange='BIST', interval=Interval.in_daily, n_bars=1000)

        #Tepeden Dönüşlerin Hesaplanması
        hh_indices = argrelextrema(data['high'].values, comparator=np.greater, order=5)[0]

        #Zigzag Noktaları için Listenin oluşturulması
        zigzag_points = []

        #Her bir tepe noktası arasındaki Vadinin bulunması
        for i in range(len(hh_indices)):
            # Add the high point
            high_idx = data.index[hh_indices[i]]
            zigzag_points.append((high_idx, data['high'].loc[high_idx], 'high'))

            if i < len(hh_indices) - 1:
                next_high_idx = data.index[hh_indices[i + 1]]
                low_range = data.loc[high_idx:next_high_idx]
                low_range = low_range[low_range.index != high_idx]
                low_range = low_range[low_range.index < next_high_idx]
                if not low_range.empty:
                    low_idx = low_range['low'].idxmin()
                    if low_idx != high_idx:
                        zigzag_points.append((low_idx, data['low'].loc[low_idx], 'low'))

        #Son verinin alınması
        last_time = data.index[-1]
        last_val = data['close'].iloc[-1]
        last_type = 'current'

        #Zigzag noktaları ile son verinin birleştirilmesi.
        zigzag_points.append((last_time, last_val, last_type))

        #Zigzag Noktalarından yeni bir dataframe oluşturulması
        df_zigzag = pd.DataFrame(zigzag_points, columns=['datetime', 'val', 'type'])

        #Oluşturulan Dataframe in tarihsel olarak sıralanması
        df_zigzag.sort_values(by='datetime', inplace=True)
        df_zigzag.reset_index(drop=True, inplace=True)

        #Son 3 verinin alınması
        df_zigzag = df_zigzag.tail(3).reset_index(drop=True)

        #Fino Seviyeleri için boş sütünlar açılması
        df_zigzag[['Fibo0', 'Fibo236', 'Fibo382', 'Fibo618', 'Fibo100']] = np.nan

        # İşlenmiş verinin en az 3 sütundan oluştuğunun kontrolü ve Fibo Seviyelerinin hesaplanması
        if len(df_zigzag) >= 3:
            df_zigzag.insert(1, 'Hisse', Hisse)
            X = df_zigzag.loc[0, 'val']
            Y = df_zigzag.loc[1, 'val']
            C = df_zigzag.loc[2,'val']

            #Eğer C Değeri X ve Y nin arasında ise
            if min(X, Y) <= C <= max(X, Y):
                Fibo_List = calc_fibo(Y, X) # Gözcü
                df_zigzag.loc[2, 'Fibo0'] = round(Fibo_List[0],2)
                df_zigzag.loc[2, 'Fibo236'] = round(Fibo_List[1],2)
                df_zigzag.loc[2, 'Fibo382'] = round(Fibo_List[2],2)
                df_zigzag.loc[2, 'Fibo618'] = round(Fibo_List[3],2)
                df_zigzag.loc[2, 'Fibo100'] = round(Fibo_List[4],2)
                df_zigzag.loc[2, 'Range'] = round(100 * (X - Y) / X, 2)

                #Son verinin Fibo seviyesinden uzaklığının ölçülmesi
                current_val = df_zigzag.loc[2, 'val']
                fibo_levels = df_zigzag.loc[2, ['Fibo0', 'Fibo236', 'Fibo382', 'Fibo618', 'Fibo100']].values
                percentages = calculate_percentage_distance(current_val, fibo_levels)
                df_zigzag.loc[2, ['Pct_Fibo0', 'Pct_Fibo236', 'Pct_Fibo382', 'Pct_Fibo618', 'Pct_Fibo100']] = percentages
                last_row = df_zigzag.iloc[-1].tolist()
                print(last_row[1:])
                df_bist.loc[len(df_bist)] = last_row
    except:
        pass

df_fibo236 = df_bist[['datetime', 'Hisse','val', 'Fibo236','Pct_Fibo236']].copy()
df_fibo382 = df_bist[['datetime', 'Hisse','val', 'Fibo382','Pct_Fibo382']].copy()
df_fibo618 = df_bist[['datetime', 'Hisse','val', 'Fibo618','Pct_Fibo618']].copy()
print(df_fibo382)
# DataFrame'lerin Yüzdelik Uzaklıkların Artan Sıraya Göre Sıralanması
df_fibo236_sorted = df_fibo236.sort_values(by='Pct_Fibo236').reset_index(drop=True)
df_fibo382_sorted = df_fibo382.sort_values(by='Pct_Fibo382').reset_index(drop=True)
df_fibo618_sorted = df_fibo618.sort_values(by='Pct_Fibo618').reset_index(drop=True)

# Sonuçların Bastırılması
print("Fibo236 Yüzdelik Uzaklıkları (Artan Sıra):")
print(df_fibo236_sorted.head(10))

print("\nFibo382 Yüzdelik Uzaklıkları (Artan Sıra):")
print(df_fibo382_sorted.head(10))

print("\nFibo618 Yüzdelik Uzaklıkları (Artan Sıra):")
print(df_fibo618_sorted.head(10))

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-ilt8dibg
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-ilt8dibg
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


['AGHOL', 438.5, 'current', 321.0, 357.93, 380.78, 417.72, 477.5, -48.75, 26.8, 18.37, 13.16, 4.74, 8.89]
['AGROT', 21.9, 'current', 15.57, 17.5, 18.7, 20.63, 23.76, -52.6, 28.9, 20.09, 14.61, 5.8, 8.49]
['AKBNK', 65.650002, 'current', 58.2, 61.13, 62.94, 65.86, 70.6, -21.31, 11.35, 6.88, 4.13, 0.32, 7.54]
['AKFGY', 2.54, 'current', 2.17, 2.28, 2.34, 2.45, 2.62, -20.74, 14.57, 10.24, 7.87, 3.54, 3.15]
['AKFYE', 24.4, 'current', 24.34, 24.68, 24.9, 25.24, 25.8, -6.0, 0.25, 1.15, 2.05, 3.44, 5.74]
['AKSEN', 43.599998, 'current', 41.38, 42.38, 42.99, 43.99, 45.6, -10.2, 5.09, 2.8, 1.4, 0.89, 4.59]
['ALARK', 103.6, 'current', 100.7, 102.4, 103.45, 105.15, 107.9, -7.15, 2.8, 1.16, 0.14, 1.5, 4.15]
['ALFAS', 68.550003, 'current', 66.1, 67.19, 67.86, 68.94, 70.7, -6.96, 3.57, 1.98, 1.01, 0.57, 3.14]
['ARDYZ', 38.360001, 'current', 34.9, 36.68, 37.78, 39.56, 42.44, -21.6, 9.02, 4.38, 1.51, 3.13, 10.64]
['ASELS', 64.599998, 'current', 55.45, 57.96, 59.52, 62.03, 66.1, -19.21, 14.16, 10.28, 7.86